## Génération d'un tableau de coordonnées fictives

In [1]:
import numpy as np

atomes_nb = 4
atomes_nb_incl_fictif = atomes_nb + 1

# Génération d'un vecteur de position des atomes d'une molécule. Pour atome, 
# le vecteur contient 3 coordonnées x, y, z, chacune dans l'intervalle [-1,1[. 

positions = np.random.random_sample(atomes_nb*3) * 2 - 1

positions

array([ 0.51066302,  0.57901401,  0.66236687, -0.65381622,  0.05543079,
        0.02551141,  0.00260623,  0.76495211, -0.25755188,  0.50067112,
        0.05163194, -0.90063594])

## Conversion des coordonnées en distances

L'objectif est de fournir au réseau de neurones un vecteur contenant chaque coordonnée x, y, z et masse m des atomes contenant la molécule, afin d'obtenir en sortie le déplacement nécessaire des atomes et d'en déduire les nouvelles positions. On ne s'intéresse ici qu'aux positions, afin d'obtenir le vecteur distances de la molécule.

Le vecteur distance contient pour chaque atome sa distance aux 4 atomes suivants (dans l'ordre donné au vecteur positions initial). Si l'atome suivant n'existe pas (on arrive à la fin de la liste), alors on impute une valeur 0.

Afin de s'assurer que l'on pourra reconstruire la molécule avec la différence de distances fournie en sortie du RN, on ajoute un atome fictif prenant comme indice 0 et qui possédera une masse nulle. Cet atome peut être placé librement, à condition qu'il n'appartienne pas au plan formé par les trois premiers atomes.

### Ajout d'un atome fictif

In [2]:
def ajout_atome_fictif(positions):
    """ Ajoute un atome fictif aux positions initiales. Pour le moment, le place simplement
    aux coordonnées (0,0,0). Dans le futur, elle le placera à une position n'appartenant pas avec
    certitude au plan formé par les trois premiers atomes. """
    return np.insert(positions, 0, [0,0,0])
    
positions = ajout_atome_fictif(positions)
positions

array([ 0.        ,  0.        ,  0.        ,  0.51066302,  0.57901401,
        0.66236687, -0.65381622,  0.05543079,  0.02551141,  0.00260623,
        0.76495211, -0.25755188,  0.50067112,  0.05163194, -0.90063594])

### Génération du vecteur distances

In [3]:
import math

def pos_to_index(i, j):
    """ Prend les coordonnées d'une entrée dans la matrice de distances et renvoie l'index de la distance
    correspondante dans le vecteur de distances. """
    return i*4+j-1

def get_atome_coord(positions, index):
    """ Renvoie les coordonnées de l'atome ayant l'index donné """
    return ((positions[3*index], positions[3*index+1], positions[3*index+2]))
    
def calcul_dist(x1, y1, z1, x2, y2, z2):
    """ Calcule et retourne la distance entre deux points selon leurs coordonnées en 3 dimensions"""

    return math.sqrt(pow(x1 - x2, 2) + pow(y1 - y2, 2) + pow(z1 - z2, 2))
    
def get_val_dist_matrice(i, j, positions):
    """ Renvoie la valeur de la matrice de distances aux coordonnées (i, j). La matrice complète n'est
    jamais calculée. """
    coord_i = get_atome_coord(positions, i)  # Coordonnées de l'atome i
    coord_j = get_atome_coord(positions, j)  # Coordonnées de l'atome j

    return calcul_dist(coord_i[0], coord_i[1], coord_i[2], coord_j[0], coord_j[1], coord_j[2])
    
def vect_distances(positions, nb_atomes):
    """ Renvoie le vecteur de distances à partir des coordonnées des atomes et du nombre d'atomes dans
    la molécule. La matrice de distances complète n'est pas générée, seules les valeurs nécessaires
    sont calculées à la demande """
    vect_distances = np.zeros(shape=((nb_atomes-1)*4))
    for j in range(nb_atomes - 1):
        k=0  # Permet d'accéder simplement à l'indice dans le tableau de sortie
        for i in range(j+1, j+5):
            if i < nb_atomes:
                vect_distances[j*4+k] = get_val_dist_matrice(i, j, positions)
            k += 1
            
    return vect_distances
            

vect_distances(positions, atomes_nb_incl_fictif)

array([1.01723341, 0.65665749, 0.80715023, 1.03173762, 1.42679226,
       1.06721375, 1.64960889, 0.        , 1.00719202, 1.48006904,
       0.        , 0.        , 1.08187403, 0.        , 0.        ,
       0.        ])

### Test de la génération du vecteur distances

In [4]:
# On génère un tableau de positions d'atomes
pos_test = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
nb_atomes_test = int(len(pos_test)/3)

# On ajoute l'atome fictif
pos_test = ajout_atome_fictif(pos_test)

# Valeurs attendues du vecteur distances
# [d10, d20, d30, d40,
#  d21, d31, d41, d51,
#  d32, d42, d52, 0,
#  d43, d53, 0,   0,
#  d54, 0,   0,   0]
#
# d10 = 3,74...
# d20 = 8,77...
# d30 = 13,92...
# d40 = 19,10...
# d21 = 5,19...
# d31 = 10,39...
# d41 = 15,58...
# d51 = 20,78...
# d32 = 5,19...
# d42 = 10,39...
# d52 = 15,58...
# d43 = 5,19...
# d53 = 10,39...
# d54 = 5,19...

distances_test = vect_distances(pos_test, nb_atomes_test+1)

# Les valeurs de retour sont bien celles attendues

## Déduction des positions en fonction des distances

La sortie du RN nous fournit un vecteur différence de distance (que l'on note delta_distances) qui correspond aux variations des distances inter-atomiques de la molécule induites par l'optimisation. La connaissance du vecteur distances fourni en entrée du RN permet de déduire le nouveau vecteur distance, et la connaissance des coordonnées de l'atome fictif permet de déduire les nouvelles positions des atomes de la molécule.

### Calcul des nouvelles distances inter-atomiques

#### Simulation de la sortie du RN
L'objectif ici est de reconstruire la molécule initiale, on suppose donc que le RN renvoie un vecteur variations de distances nul

In [5]:
def simulation_rn(distances_init, atomes_nb):
    return np.zeros(shape=(atomes_nb*4,))

# On simule l'appel au RN pour obtenir le vecteur variations de distances
delta_distances = simulation_rn(distances_test, nb_atomes_test)

#### Construction du nouveau vecteur positions

In [7]:
def reconstruction_positions(distances_init, delta_distances, pos_at_fictif):
    """ Fonction calculant les nouvelles positions des atomes de la molécule à partir des distances initiales,
    des variations de distances fournies par le RN et de la position initiale de l'atome fictif"""
    pass
    


In [8]:
from sympy import *
from sympy.solvers import nsolve
from sympy import N
from sympy import Symbol
from sympy import sqrt
from sympy import Eq
import mpmath

mpmath.mp.dps = 15

x2 = Symbol('x2')
y2 = Symbol('y2')
z2 = Symbol('z2')

x0 = 0
y0 = 0
z0 = 0

x1 = 3.74
y1 = 0
z1 = 0

d20 = 8.77
d21 = 5.19

y2=1000


sol = solve((x2-x0)**2 + (y2-y0)**2 + (z2-z0)**2 + (x2-x1)**2 + (y2-y1)**2 + (z2-z1)**2 - d20**2 -d21**2, x2)
sol

[-0.02*sqrt(-2500.0*z2**2 - 2499878931.0) + 1.87,
 0.02*sqrt(-2500.0*z2**2 - 2499878931.0) + 1.87]

#### Tests des formules de calcul des points sur plusieurs exemples

In [16]:

def aff_point(idPoint, x, y, z):
    print("a"+str(idPoint)+" : ("+str(x)+", "+str(y)+", "+str(z)+")")
    
def aff_distance(id_pt1, id_pt2, x1, y1, z1, x2, y2, z2):
    print("d"+str(id_pt1)+str(id_pt2)+" = "+str(calcul_dist(x1, y1, z1, x2, y2, z2)))

def translation(x_ori, y_ori, z_ori, x_vect, y_vect, z_vect):
    return (x_ori+x_vect, y_ori+y_vect, z_ori+z_vect)
  

# Coordonnées d'origine
x_init0 = 0
y_init0 = 0
z_init0 = 1

x_init1 = 2
y_init1 = 4
z_init1 = 6

x_init2 = 4
y_init2 = -6
z_init2 = 8

x_init3 = 1
y_init3 = 1
z_init3 = 1

# Calcul des distances entre les coordonnées d'origine
d10 = calcul_dist(x_init0, y_init0, z_init0, x_init1, y_init1, z_init1)
d20 = calcul_dist(x_init0, y_init0, z_init0, x_init2, y_init2, z_init2)
d21 = calcul_dist(x_init1, y_init1, z_init1, x_init2, y_init2, z_init2)
d30 = calcul_dist(x_init0, y_init0, z_init0, x_init3, y_init3, z_init3)
d31 = calcul_dist(x_init1, y_init1, z_init1, x_init3, y_init3, z_init3)
d32 = calcul_dist(x_init2, y_init2, z_init2, x_init3, y_init3, z_init3)
 



# Calcul du nouveau a0 en le considérant à l'origine
x0 = 0
y0 = 0
z0 = 0

# Calcul du nouveau a1 sachant que a0 est à l'origine
x1 = d10
y1 = 0
z1 = 0


# Calcul du nouveau a2 sachant que a0 est à l'origine
x2 = (d20**2-d21**2+d10**2)/(2*d10)
y2 = math.sqrt(d20**2-((d20**2-d21**2+d10**2)/(2*d10))**2)
z2 = 0


# Calcul du nouveau a3 sachant que a0 est à l'origine
x3 = (d30**2+d10**2-d31**2)/(2*d10)
y3 = (-2*x3*x2+d20**2-d32**2+d30**2)/(2*y2)
z3 = math.sqrt( -x3**2 - y3**2 + d30**2)


# Translation de tous les points calculés selon le vrai a0
pos_0 = translation(x0, y0, z0, x_init0, y_init0, z_init0)
x0 = pos_0[0]
y0 = pos_0[1]
z0 = pos_0[2]

pos_1 = translation(x1, y1, z1, x_init0, y_init0, z_init0)
x1 = pos_1[0]
y1 = pos_1[1]
z1 = pos_1[2]

pos_2 = translation(x2, y2, z2, x_init0, y_init0, z_init0)
x2 = pos_2[0]
y2 = pos_2[1]
z2 = pos_2[2]

pos_3 = translation(x3, y3, z3, x_init0, y_init0, z_init0)
x3 = pos_3[0]
y3 = pos_3[1]
z3 = pos_3[2]


print()
print("Affichage des distances pré-traitement : ")
aff_distance(1, 0, x_init0, y_init0, z_init0, x_init1, y_init1, z_init1)
aff_distance(2, 0, x_init0, y_init0, z_init0, x_init2, y_init2, z_init2)
aff_distance(2, 1, x_init1, y_init1, z_init1, x_init2, y_init2, z_init2)
aff_distance(3, 0, x_init0, y_init0, z_init0, x_init3, y_init3, z_init3)
aff_distance(3, 1, x_init1, y_init1, z_init1, x_init3, y_init3, z_init3)
aff_distance(3, 2, x_init2, y_init2, z_init2, x_init3, y_init3, z_init3)


print()
print("Affichage des distances post-traitement : ")

aff_distance(1,0, x0, y0, z0, x1, y1, z1)
aff_distance(2,0, x0, y0, z0, x2, y2, z2)
aff_distance(2,1, x1, y1, z1, x2, y2, z2)
aff_distance(3, 0, x0, y0, z0, x3, y3, z3)
aff_distance(3, 1, x1, y1, z1, x3, y3, z3)
aff_distance(3, 2, x2, y2, z2, x3, y3, z3)


print()
print("Affichage des points : ")

aff_point(0, x0, y0, z0)
aff_point(1, x1, y1, z1)
aff_point(2, x2, y2, z2)
aff_point(3, x3, y3, z3)







Affichage des distances pré-traitement : 
d10 = 6.708203932499369
d20 = 10.04987562112089
d21 = 10.392304845413264
d30 = 1.4142135623730951
d31 = 5.916079783099616
d32 = 10.344080432788601

Affichage des distances post-traitement : 
d10 = 6.708203932499369
d20 = 10.049875621120892
d21 = 10.392304845413264
d30 = 1.4142135623730951
d31 = 5.916079783099616
d32 = 10.344080432788603

Affichage des points : 
a0 : (0, 0, 1)
a1 : (6.708203932499369, 0, 1)
a2 : (2.832352771499734, 9.642498523607758, 1)
a3 : (0.8944271909999164, -0.47014094139961465, 1.9894278625649693)


In [12]:
# Coordonnées d'origine
x_init0 = 0
y_init0 = 0
z_init0 = 1

x_init2 = 0.74905055
y_init2 = 0.08824051
z_init2 = -0.96609335

x_init1 = 0.32891684
y_init1 = -0.34937679
z_init1 = 0.04682129

x_init3 = 0.38808914
y_init3 = 0.97792745
z_init3 = 0.57889907


# Calcul des distances entre les coordonnées d'origine
d10 = calcul_dist(x_init0, y_init0, z_init0, x_init1, y_init1, z_init1)
d20 = calcul_dist(x_init0, y_init0, z_init0, x_init2, y_init2, z_init2)
d21 = calcul_dist(x_init1, y_init1, z_init1, x_init2, y_init2, z_init2)
d30 = calcul_dist(x_init0, y_init0, z_init0, x_init3, y_init3, z_init3)
d31 = calcul_dist(x_init1, y_init1, z_init1, x_init3, y_init3, z_init3)
d32 = calcul_dist(x_init2, y_init2, z_init2, x_init3, y_init3, z_init3)
 



# Calcul du nouveau a0 en le considérant à l'origine
x0 = 0
y0 = 0
z0 = 0

# Calcul du nouveau a1 sachant que a0 est à l'origine
x1 = d10
y1 = 0
z1 = 0


# Calcul du nouveau a2 sachant que a0 est à l'origine
x2 = (d20**2-d21**2+d10**2)/(2*d10)
y2 = math.sqrt(d20**2-((d20**2-d21**2+d10**2)/(2*d10))**2)
z2 = 0


# Calcul du nouveau a3 sachant que a0 est à l'origine
x3 = (d30**2+d10**2-d31**2)/(2*d10)
y3 = (-2*x3*x2+d20**2-d32**2+d30**2)/(2*y2)
z3 = math.sqrt( -x3**2 - y3**2 + d30**2)


# Translation de tous les points calculés selon le vrai a0
pos_0 = translation(x0, y0, z0, x_init0, y_init0, z_init0)
x0 = pos_0[0]
y0 = pos_0[1]
z0 = pos_0[2]

pos_1 = translation(x1, y1, z1, x_init0, y_init0, z_init0)
x1 = pos_1[0]
y1 = pos_1[1]
z1 = pos_1[2]

pos_2 = translation(x2, y2, z2, x_init0, y_init0, z_init0)
x2 = pos_2[0]
y2 = pos_2[1]
z2 = pos_2[2]

pos_3 = translation(x3, y3, z3, x_init0, y_init0, z_init0)
x3 = pos_3[0]
y3 = pos_3[1]
z3 = pos_3[2]


print()
print("Affichage des distances pré-traitement : ")
aff_distance(1, 0, x_init0, y_init0, z_init0, x_init1, y_init1, z_init1)
aff_distance(2, 0, x_init0, y_init0, z_init0, x_init2, y_init2, z_init2)
aff_distance(2, 1, x_init1, y_init1, z_init1, x_init2, y_init2, z_init2)
aff_distance(3, 0, x_init0, y_init0, z_init0, x_init3, y_init3, z_init3)
aff_distance(3, 1, x_init1, y_init1, z_init1, x_init3, y_init3, z_init3)
aff_distance(3, 2, x_init2, y_init2, z_init2, x_init3, y_init3, z_init3)


print()
print("Affichage des distances post-traitement : ")

aff_distance(1,0, x0, y0, z0, x1, y1, z1)
aff_distance(2,0, x0, y0, z0, x2, y2, z2)
aff_distance(2,1, x1, y1, z1, x2, y2, z2)
aff_distance(3, 0, x0, y0, z0, x3, y3, z3)
aff_distance(3, 1, x1, y1, z1, x3, y3, z3)
aff_distance(3, 2, x2, y2, z2, x3, y3, z3)


print()
print("Affichage des points : ")

aff_point(0, x0, y0, z0)
aff_point(1, x1, y1, z1)
aff_point(2, x2, y2, z2)
aff_point(3, x3, y3, z3)




Affichage des distances pré-traitement : 
d10 = 1.0671457642813158
d20 = 2.1057982275077034
d21 = 1.1806850991962181
d30 = 1.1332613428932918
d31 = 1.4312039234780614
d32 = 1.8190210467647723

Affichage des distances post-traitement : 
d10 = 1.0671457642813158
d20 = 2.1057982275077034
d21 = 1.180685099196218
d30 = 1.1332613428932918
d31 = 1.4312039234780614
d32 = 1.8190210467647723

Affichage des points : 
a0 : (0, 0, 1)
a1 : (1.0671457642813158, 0, 1)
a2 : (1.9581059559134713, 0.7747304307891719, 1)
a3 : (0.17557895813475674, 1.1114998210602476, 1.1342439888399407)


In [64]:
type(1 + 8.42936970217881e-8*I)

sympy.core.add.Add